# Simulation of investment decisions

The following notebook analyzes various fund investment decisions.

The main goal of this analysis is to see the performance of the best fund in the "stock oriented" funds compared to a random selection (out of the funds in that specific group).

In [44]:
# Import required libraries
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

# Read the data file
investments_raw = pd.read_csv("../data/all_gemel_data_raw.csv")

# glimpse on the head of the data
investments_raw.head()

,FUND_ID,FUND_NAME,FUND_CLASSIFICATION,CONTROLLING_CORPORATION,MANAGING_CORPORATION,REPORT_PERIOD,INCEPTION_DATE,TARGET_POPULATION,SPECIALIZATION,SUB_SPECIALIZATION,...,STANDARD_DEVIATION,ALPHA,SHARPE_RATIO,LIQUID_ASSETS_PERCENT,STOCK_MARKET_EXPOSURE,FOREIGN_EXPOSURE,FOREIGN_CURRENCY_EXPOSURE,MANAGING_CORPORATION_LEGAL_ID,CURRENT_DATE,...33
0,12535,"""אינטרגמל השתלמות"" קרן השתלמות לשכירים ולעצמאי...",קרנות השתלמות,"י.ד מור השקעות בע""מ","מור קופות גמל בע""מ",201812.0,26/12/2018,כלל האוכלוסיה,כללי,כללי,...,NaN,NaN,NaN,93.4,0.05,0.02,0.02,514956465.0,29/05/2019,NaN
1,12536,"""אינטרגמל השתלמות"" קרן השתלמות לשכירים ולעצמאי...",קרנות השתלמות,"י.ד מור השקעות בע""מ","מור קופות גמל בע""מ",201812.0,26/12/2018,כלל האוכלוסיה,מניות,מניות,...,NaN,NaN,NaN,93.7,0.12,0.05,0.05,514956465.0,29/05/2019,NaN
2,9676,"""אינטרגמל השתלמות"" קרן השתלמות לשכירים ולעצמאי...",קרנות השתלמות,"אס.או.אי אחזקות בע""מ","אינטרגמל קופות גמל בע""מ",201510.0,14/10/2015,כלל האוכלוסיה,כללי,כללי,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,514956465.0,10/07/2018 08:31:02,7137.0
3,9676,"""אינטרגמל השתלמות"" קרן השתלמות לשכירים ולעצמאי...",קרנות השתלמות,"אס.או.אי אחזקות בע""מ","אינטרגמל קופות גמל בע""מ",201511.0,14/10/2015,כלל האוכלוסיה,כללי,כללי,...,NaN,NaN,NaN,NaN,0.02,0.00,0.00,514956465.0,10/07/2018 08:31:02,7138.0
4,9676,"""אינטרגמל השתלמות"" קרן השתלמות לשכירים ולעצמאי...",קרנות השתלמות,"אס.או.אי אחזקות בע""מ","אינטרגמל קופות גמל בע""מ",201512.0,14/10/2015,כלל האוכלוסיה,כללי,כללי,...,NaN,NaN,NaN,99.2,0.03,0.00,0.00,514956465.0,10/07/2018 08:31:02,7139.0


In [18]:
# show the columns
investments_raw.columns

Index(['FUND_ID', 'FUND_NAME', 'FUND_CLASSIFICATION',
       'CONTROLLING_CORPORATION', 'MANAGING_CORPORATION', 'REPORT_PERIOD',
       'INCEPTION_DATE', 'TARGET_POPULATION', 'SPECIALIZATION',
       'SUB_SPECIALIZATION', 'DEPOSITS', 'WITHDRAWLS', 'INTERNAL_TRANSFERS',
       'NET_MONTHLY_DEPOSITS', 'TOTAL_ASSETS', 'AVG_ANNUAL_MANAGEMENT_FEE',
       'AVG_DEPOSIT_FEE', 'MONTHLY_YIELD', 'YEAR_TO_DATE_YIELD',
       'YIELD_TRAILING_3_YRS', 'YIELD_TRAILING_5_YRS',
       'AVG_ANNUAL_YIELD_TRAILING_3YRS', 'AVG_ANNUAL_YIELD_TRAILING_5YRS',
       'STANDARD_DEVIATION', 'ALPHA', 'SHARPE_RATIO', 'LIQUID_ASSETS_PERCENT',
       'STOCK_MARKET_EXPOSURE', 'FOREIGN_EXPOSURE',
       'FOREIGN_CURRENCY_EXPOSURE', 'MANAGING_CORPORATION_LEGAL_ID',
       'CURRENT_DATE', '...33'],
      dtype='object')

We're going to focus on the specific tool called "קרנות השתלמות" (i.e., FUND_CLASSIFICATION == "קרנות השתלמות"), and within it, just the funds which invest mostly in stocks (SPECIALIZATION == "מניות" and SUB_SPECIALIZATION == "מניות").

In [77]:
# filter to funds with stock specialization
stock_specialization = investments_raw[investments_raw.SPECIALIZATION == "מניות"]
stock_specialization = stock_specialization[stock_specialization.FUND_CLASSIFICATION == "קרנות השתלמות"]
stock_specialization = stock_specialization[stock_specialization.SUB_SPECIALIZATION == "מניות"]
stock_specialization

,FUND_ID,FUND_NAME,FUND_CLASSIFICATION,CONTROLLING_CORPORATION,MANAGING_CORPORATION,REPORT_PERIOD,INCEPTION_DATE,TARGET_POPULATION,SPECIALIZATION,SUB_SPECIALIZATION,...,STANDARD_DEVIATION,ALPHA,SHARPE_RATIO,LIQUID_ASSETS_PERCENT,STOCK_MARKET_EXPOSURE,FOREIGN_EXPOSURE,FOREIGN_CURRENCY_EXPOSURE,MANAGING_CORPORATION_LEGAL_ID,CURRENT_DATE,...33
1,12536,"""אינטרגמל השתלמות"" קרן השתלמות לשכירים ולעצמאי...",קרנות השתלמות,"י.ד מור השקעות בע""מ","מור קופות גמל בע""מ",201812.0,26/12/2018,כלל האוכלוסיה,מניות,מניות,...,NaN,NaN,NaN,93.7,0.12,0.05,0.05,514956465.0,29/05/2019,NaN
798,1446,אומגה קרן השתלמות מסלול מניות,קרנות השתלמות,"מנורה מבטחים החזקות בע""מ",מנורה מבטחים והסתדרות המהנדסים ניהול קופות גמל...,201312.0,03/02/2009,עובדי סקטור מסויים,מניות,מניות,...,NaN,NaN,NaN,94.1,7.75,3.30,3.31,520027715.0,10/07/2018 08:29:21,5656.0
799,1446,אומגה קרן השתלמות מסלול מניות,קרנות השתלמות,"מנורה מבטחים החזקות בע""מ",מנורה מבטחים והסתדרות המהנדסים ניהול קופות גמל...,201401.0,03/02/2009,עובדי סקטור מסויים,מניות,מניות,...,NaN,NaN,NaN,93.8,7.67,3.21,3.22,520027715.0,10/07/2018 08:30:12,4713.0
800,1446,אומגה קרן השתלמות מסלול מניות,קרנות השתלמות,"מנורה מבטחים החזקות בע""מ",מנורה מבטחים והסתדרות המהנדסים ניהול קופות גמל...,201402.0,03/02/2009,עובדי סקטור מסויים,מניות,מניות,...,NaN,NaN,NaN,93.9,7.99,3.36,3.37,520027715.0,10/07/2018 08:30:12,4714.0
801,1446,אומגה קרן השתלמות מסלול מניות,קרנות השתלמות,"מנורה מבטחים החזקות בע""מ",מנורה מבטחים והסתדרות המהנדסים ניהול קופות גמל...,201403.0,03/02/2009,עובדי סקטור מסויים,מניות,מניות,...,NaN,NaN,NaN,93.6,7.85,3.14,3.15,520027715.0,10/07/2018 08:30:12,4715.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91660,1015,"תמיר פישמן השתלמות מניות בישראל ובחו""ל",קרנות השתלמות,אחר,"תמיר פישמן גמל והשתלמות בע""מ",201210.0,29/07/2004,כלל האוכלוסיה,מניות,מניות,...,19.30,-4.20,-0.14,87.6,14.86,6.94,7.12,513571273.0,10/07/2018 08:28:19,3055.0
91661,1015,"תמיר פישמן השתלמות מניות בישראל ובחו""ל",קרנות השתלמות,אחר,"תמיר פישמן גמל והשתלמות בע""מ",201211.0,29/07/2004,כלל האוכלוסיה,מניות,מניות,...,19.14,-3.96,-0.08,87.4,15.36,7.58,7.78,513571273.0,10/07/2018 08:28:19,3056.0
91662,1015,"תמיר פישמן השתלמות מניות בישראל ובחו""ל",קרנות השתלמות,אחר,"תמיר פישמן גמל והשתלמות בע""מ",201212.0,29/07/2004,כלל האוכלוסיה,מניות,מניות,...,19.14,-3.00,-0.08,87.8,15.97,7.86,8.07,513571273.0,10/07/2018 08:28:19,3057.0
91663,1015,"תמיר פישמן השתלמות מניות בישראל ובחו""ל",קרנות השתלמות,אחר,"תמיר פישמן גמל והשתלמות בע""מ",201301.0,29/07/2004,כלל האוכלוסיה,מניות,מניות,...,18.54,-2.40,0.02,87.0,16.02,8.17,8.40,513571273.0,10/07/2018 08:29:21,3286.0


I want to verify the minimum date for each of the funds, and the number of observations for each fund.

In [104]:
# Number of observations per fund
obs_per_fund = (stock_specialization
   .groupby("FUND_NAME")
   .size()
   .sort_values(ascending=False))

obs_per_fund

FUND_NAME
אקסלנס השתלמות מניות                                                200
מנורה מבטחים השתלמות מסלול מניות                                    176
מיטב דש השתלמות מניות                                               142
רום קרן השתלמות לעובדי הרשויות המקומיות מסלול ספיר מניות            142
קרן השתלמות להנדסאים וטכנאים - מסלול מניות                          142
קרן החיסכון לצבא הקבע מניות                                         142
כלל השתלמות מניות                                                   142
קרן השתלמות למשפטנים מסלול מנייתי                                   142
קרן השתלמות לאקדמאים במדעי החברה והרוח מסלול מניות                  142
מינהל - קרן ההשתלמות לפקידים עובדי המינהל והשרותים - מסלול מניות    142
רעות קרן השתלמות - מניות                                            142
אנליסט השתלמות מניות                                                142
הלמן-אלדובי השתלמות מניות                                           130
פסגות שיא השתלמות מניות                               

Let's check the minimum, maximum, and number of observations per each fund, and try to understand why the number of observations varies so signficantly.

In [149]:
min_max_date_per_fund = (stock_specialization[["FUND_NAME", "REPORT_PERIOD"]]
   .groupby("FUND_NAME")
   .agg(['min', 'max', "size"])
   .reset_index())

min_max_date_per_fund.columns = ['_'.join(col).strip() for col in min_max_date_per_fund.columns.values]

min_max_date_per_fund

,FUND_NAME_,REPORT_PERIOD_min,REPORT_PERIOD_max,REPORT_PERIOD_size
0,"""אינטרגמל השתלמות"" קרן השתלמות לשכירים ולעצמאי...",201812.0,201812.0,1
1,אומגה קרן השתלמות מסלול מניות,201312.0,202210.0,107
2,אי.בי.אי קרן השתלמות מסלול מניות,201101.0,201812.0,96
3,איילון השתלמות מסלול מניות,201308.0,201701.0,42
4,איפקס השתלמות מניות,201201.0,201406.0,30
5,אלטשולר שחם השתלמות מניות,201206.0,202210.0,125
6,אנליסט השתלמות מניות,201101.0,202210.0,142
7,אקסלנס השתלמות זהב - מסלול מניות,201101.0,201312.0,36
8,אקסלנס השתלמות מניות,201101.0,202012.0,200
9,דש אקונומיסט השתלמות מניות,201307.0,201406.0,12


The number of observations varies significantly. 
It seems that in order to increase the sample size we should not start from 2011 but maybe from 2015 (this will help add some funds).
Also, verify why there are two funds with too many observations (duplicates?): אקסלנס השתלמות מניות (n=200) and מנורה מבטחים השתלמות מניות (n=176) versus the rest with history from 2011 (n=142, i.e., 12 years * 12 months - 2 months since we end on October's data).

In addition, the subclassification doesn't seem to change over time.

In [167]:
# Is subclassification consistnet?
check_consistent_subclass = (investments_raw
   .query('SPECIALIZATION == "מניות"')
   .query('FUND_CLASSIFICATION == "קרנות השתלמות"')
   .filter(items = ["FUND_NAME", "SUB_SPECIALIZATION"])
   .groupby("FUND_NAME")
   .nunique("SUB_SPECIALIZATION")
   .sort_values(by = "SUB_SPECIALIZATION")
   .reset_index()
)

check_consistent_subclass

,FUND_NAME,SUB_SPECIALIZATION
0,"""אינטרגמל השתלמות"" קרן השתלמות לשכירים ולעצמאי...",1
1,מנורה מבטחים השתלמות מחקה מדד S&P500,1
2,מינהל - קרן ההשתלמות לפקידים עובדי המינהל והשר...,1
3,מילניום קרן השתלמות מסלולית - מסלול מניות,1
4,מיטב וויזר השתלמות מניות,1
...,...,...
66,הדס ארזים אפסילון השתלמות מסלול מנייתי,1
67,דש השתלמות מניות,1
68,דש אקונומיסט השתלמות מניות,1
69,הלמן-אלדובי קרן השתלמות מסלול פאסיבי - מדדי מנ...,1


Extract the fund names we will work on from now on.

In [214]:
min_date_check = 201500
expected_months = (round(202310/100) - round(min_date_check/100))*12 - 2

use_funds_pre = (stock_specialization
   .filter(items = ["FUND_NAME", "REPORT_PERIOD"])
   .query(f"REPORT_PERIOD >= {min_date_check}")
   .groupby("FUND_NAME")
   .agg(['min', 'max', "size"])
   .reset_index())

use_funds_pre.columns = [''.join(col).strip() for col in min_max_date_per_fund.columns.values]

use_funds = (use_funds_pre
   .rename(columns = {"FUND_NAME_" : "FUND_NAME"})
   .query("REPORT_PERIOD_max == 202210")
   .query(f"REPORT_PERIOD_min >= {min_date_check}")
   .query(f"REPORT_PERIOD_size == {expected_months}"))

print(use_funds["FUND_NAME"].count())

fund_name_list = list(use_funds["FUND_NAME"])

fund_name_list

16


['אומגה קרן השתלמות מסלול מניות',
 'אלטשולר שחם השתלמות מניות',
 'אנליסט השתלמות מניות',
 'הראל השתלמות מסלול מניות',
 'ילין לפידות קרן השתלמות מסלול מניות',
 'כלל השתלמות מניות',
 'מגדל השתלמות מסלול מניות',
 'מיטב דש השתלמות מניות',
 'מינהל - קרן ההשתלמות לפקידים עובדי המינהל והשרותים - מסלול מניות',
 'מנורה מבטחים השתלמות מסלול מניות',
 'קרן החיסכון לצבא הקבע מניות',
 'קרן השתלמות לאקדמאים במדעי החברה והרוח מסלול מניות',
 'קרן השתלמות להנדסאים וטכנאים - מסלול מניות',
 'קרן השתלמות למשפטנים מסלול מנייתי',
 'רום קרן השתלמות לעובדי הרשויות המקומיות מסלול ספיר מניות',
 'רעות קרן השתלמות - מניות']

The analysis shows that choosing stock funds which started at 2015 and have statistics up until October 2022, yields 16 funds that we can work with.

We will now define a function that given a start date and an end date computes the yield of all of the funds, returns the best fund's yield, the best fund's name, and the overall average.